In [1]:
import requests, datetime, pickle

URL = 'https://open-api.bingx.com'

descargar_data = False

# Obtener la serie de precios para BTC, ETH y XRP 

En temporalidades: 1d y/o 4h, 2h, 1h, 30m, 15m

data_raw = {<br />
    'BTC': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'ETH': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'XRP':<br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    }<br />
}<br />

In [2]:
from backtesting.fechas import past_timestamp, now_timestamp, retrasar_tres_horas
from backtesting.data import response_to_dataframe
from backtesting.ficheros import crear_directorio

In [3]:
if not descargar_data:
    with open('./data/data_raw.pickle', 'rb') as f:
        data_raw = pickle.load(f)
    activos = list(data_raw.keys())
    temporalidades = list(data_raw[activos[0]].keys())

else:
    path = '/openApi/swap/v2/quote/klines'

    data_1d = True
    data_low_temporality = True

    temporalidades = []
    if data_1d:
        temporalidades.append('1d')
    if data_low_temporality:
        temporalidades.append('4h')
        temporalidades.append('2h')
        temporalidades.append('1h')
        temporalidades.append('30m')
        temporalidades.append('15m')
    if not data_1d and not data_low_temporality:
        exit()

    now = datetime.datetime.now()
    start_time = []
    for t in temporalidades:
        if t == '1d':
            start_time.append(int(past_timestamp(365, 'days', now)))
        elif t == '4h':
            start_time.append(int(past_timestamp(45, 'days', now)))
        elif t == '2h':
            start_time.append(int(past_timestamp(20, 'days', now)))
        elif t == '1h':
            start_time.append(int(past_timestamp(20, 'days', now)))
        elif t == '30m':
            start_time.append(int(past_timestamp(6, 'days', now)))
        elif t == '15m':
            start_time.append(int(past_timestamp(3, 'days', now)))

    limit = 555
    activos = ['BTC', 'ETH', 'XRP']
    service = URL + path
    endTime = int(datetime.datetime.timestamp(datetime.datetime.now()) * 1000)
    data_raw = {}
    for activo in activos:
        data_raw[activo] = {}
        symbol = activo + '-USDT'
        for i, interval in enumerate(temporalidades):
            response = requests.get(service, params={'symbol':symbol, 'interval':interval, 'limit':limit, 'startTime':start_time[i], 'endTime':endTime})
            df = response_to_dataframe(response.json())
            if df is None:
                continue
            df['time'] = df['time'].apply(retrasar_tres_horas)
            df = df.set_index('time')
            data_raw[activo][interval] = df

    crear_directorio('data')
    with open('./data/data_raw.pickle', 'wb') as f:
        pickle.dump(data_raw, f, pickle.HIGHEST_PROTOCOL)

# Aplicar Squeeze Momentum Indicator a cada serie temporal y obtener la temporalidad que tenga mayor correlacion para cada activo

SMI_default = {<br />
    'BTC': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'ETH': <br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    },<br />
    'XRP':<br />
    {'1d': pd.DataFrame(), '4h': pd.DataFrame(), '2h': pd.DataFrame(), '1h': pd.DataFrame(), '30m': pd.DataFrame(), '15m': pd.<br />DataFrame()
    }<br />
<br />
}<br />
<br />
SMI_default_corr = {<br />
    'BTC': <br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    },<br />
    'ETH': <br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    },<br />
    'XRP':<br />
    {'1d': 0, '4h': 0, '2h': 0, '1h': 0, '30m': 0, '15m': 0<br />
    }<br />
<br />
}<br />

In [4]:
from backtesting.indicadores import squeeze_momentum_indicator
from scipy.stats import pearsonr

In [5]:
SMI_default = {}
for activo in activos:
    SMI_default[activo] = {}
    for t in temporalidades:
        df = data_raw[activo][t].iloc[:]
        df = squeeze_momentum_indicator(df)
        SMI_default[activo][t] = df.dropna()
        

In [6]:
SMI_default_corr = {}
for activo in activos:
    SMI_default_corr[activo] = {}
    for t in temporalidades:
        df = SMI_default[activo][t].iloc[:]
        df = df[['value', 'close']]
        SMI_default_corr[activo][t], _ = pearsonr(df['value'], df['close'])

print('Correlaciones')
for activo in activos:
    for t in temporalidades:
        print(f'{activo} {t} {round(SMI_default_corr[activo][t], 2)}')
    print()

Correlaciones
BTC 1d 0.36
BTC 4h 0.57
BTC 2h 0.45
BTC 1h 0.35
BTC 30m 0.07
BTC 15m 0.71

ETH 1d 0.41
ETH 4h 0.47
ETH 2h 0.5
ETH 1h 0.5
ETH 30m 0.21
ETH 15m 0.65

XRP 1d 0.41
XRP 4h 0.34
XRP 2h 0.39
XRP 1h 0.27
XRP 30m 0.05
XRP 15m 0.63



In [7]:
temporalidades_filtradas = {}
for activo in activos:
    temporalidades_filtradas[activo] = {}
    for t in temporalidades:
        if SMI_default_corr[activo][t] > 0.5:
            temporalidades_filtradas[activo][t] = SMI_default_corr[activo][t]

# print('Temporalidades filtradas')
# for activo in activos:
#     for t in temporalidades:
#         print(f'{activo} {round(temporalidades_filtradas[activo][t], 2)}')
#     print()

print('Temporalidades filtradas')
for activo in temporalidades_filtradas:
    print(activo)
    for t in temporalidades_filtradas[activo]:
        print(f'correlacion {t}: {round(temporalidades_filtradas[activo][t], 2)}')
    print()

Temporalidades filtradas
BTC
correlacion 4h: 0.57
correlacion 15m: 0.71

ETH
correlacion 2h: 0.5
correlacion 1h: 0.5
correlacion 15m: 0.65

XRP
correlacion 15m: 0.63



### conclusiones 1:

- Si optimizo los parámetros para 1d, como afectan las demás temporalidades?
- Si optimizo los parámetros para una temporalidad con alta correlación, como afectan las demás temporalidades?
- Que diferencia existe entre aplicar la estrategia con parámetros por default y optimizados?

# Aplicar las condiciones de la estrategia a las temporalidades más correlacionadas y visualizar las últimas entradas

In [8]:
from backtesting.indicadores import backtest_entry

In [9]:
entradas_backtest = {}
for activos in temporalidades_filtradas:
    for t in temporalidades_filtradas[activos]:
        entrada_short, entrada_long = backtest_entry(SMI_default[activos][t])
        dict_key = activos + '_' + t
        if len(entrada_short) > 0:
            dict_key_2 = dict_key + '_short'
            entradas_backtest.update({dict_key_2: entrada_short})
        if len(entrada_long) > 0:
            dict_key_2 = dict_key + '_long'
            entradas_backtest.update({dict_key_2: entrada_long})
        

In [34]:
# tomamos las 10 últimas 
entradas_backtest['BTC_4h_short']
#entradas_backtest['BTC_15m_long']
entradas_backtest['ETH_2h_long']
entradas_backtest.keys()

dict_keys(['BTC_4h_short', 'BTC_15m_long', 'ETH_2h_short', 'ETH_2h_long', 'ETH_1h_short', 'ETH_1h_long', 'ETH_15m_short', 'ETH_15m_long', 'XRP_15m_short'])

In [31]:
entradas_backtest['ETH_2h_short']

,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
time,,,,,,,,,,,,
2023-05-03 21:00:00,1904.45,1901.87,1905.85,1895.61,22826.12,1901.579037,1830.124963,10.24,1897.626369,1834.077631,27.264018,False


In [32]:
entradas_backtest['ETH_1h_long']

,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
time,,,,,,,,,,,,
2023-04-30 03:00:00,1904.45,1904.30,1906.57,1902.65,4561.75,1912.754179,1892.499821,3.92,1912.649763,1892.604237,-2.597686,False
2023-05-01 19:00:00,1825.10,1826.73,1834.00,1821.81,9819.59,1863.756701,1815.361299,12.19,1862.319800,1816.798200,-16.004964,False
2023-05-08 19:00:00,1839.75,1846.82,1846.90,1838.67,8563.36,1877.407811,1826.431189,8.23,1875.383811,1828.455189,-10.882082,False


In [33]:
entradas_backtest['ETH_1h_short']

,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
time,,,,,,,,,,,,
2023-05-07 17:00:00,1912.96,1917.79,1918.09,1912.95,9191.2,1924.847053,1888.759947,5.14,1923.127965,1890.479035,14.793754,False
2023-05-10 21:00:00,1840.76,1831.16,1841.71,1826.87,17985.6,1879.354212,1818.598788,14.84,1877.143245,1820.809755,1.299500,False


In [35]:
entradas_backtest['XRP_15m_short']

,open,close,high,low,volume,upper_BB,lower_BB,tr,upper_KC,lower_KC,value,squeeze_on
time,,,,,,,,,,,,
2023-05-09 12:15:00,0.4225,0.423,0.4234,0.4225,2283521.0,0.426656,0.421424,0.0009,0.426592,0.421488,0.000821,False


In [25]:
## LO QUE HICE EN ESTA CELDA FUE CORROBORAR QUE SE CUMPLA EL CRITERIOR DE SQUEEZE
## MIENTRAS CORROBORO LAS SEÑALES EN TRADING VIEW HE NOTADO QUE LA CONDICION DE SQUEEZE
## NO SE CUMPLA TAL CUAL DE FORMA GRAFICA, POR LO QUE ESTOY REVISANDO QUE SE CUMPLA LOGICAMENTE
## Y SI SE CUMPLE

# # chequear que en esta fecha
# import pandas as pd
# cuatro_horas_antes_e1 = pd.to_datetime(entradas_backtest['BTC_4h_short'].index[0]) - datetime.timedelta(hours=4)
# # change to str
# cuatro_horas_antes_e1 = str(cuatro_horas_antes_e1)

# SMI_default['BTC']['4h'].loc[cuatro_horas_antes_e1:entradas_backtest['BTC_4h_short'].index[-1]]#:entradas_backtest['BTC_4h_short'].index[0]


# Optimizar los parámetros del SMI para cada activo en la temporalidad de mayor correlación

SMI_optimized = {<br />
    'BTC': <br />
    {'best_temporality': pd.DataFrame()},<br />
    'ETH': <br />
    {'best_temporality': pd.DataFrame()},<br />
    'XRP':<br />
    {'best_temporality': pd.DataFrame()}<br />
<br />
}<br />

# Aplicar los criterios de la estrategia para cada configuración del indicador y visualizar las señales